# Prompt Engineering

Modified from the file `Lab_Prompt_Engineering.ipynb`

#### _Goal: See how even small amounts of data can make an LLM much more useful!_


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/dcai_lab/dcai-lab/prompt_engineering/

/content/drive/MyDrive/dcai_lab/dcai-lab/prompt_engineering


In [3]:
!pwd

/content/drive/MyDrive/dcai_lab/dcai-lab/prompt_engineering


In [4]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 2.7 MB/s eta 0:00:00


In [5]:
import os
from groq import Groq
from google.colab import userdata

In [6]:
client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)

In [7]:
def get_completion(llm_prompt, model, options={}):
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content":llm_prompt},
        ],
        model=model,
        **options
    )
    completion = response.choices[0].message.content
    return completion

### Scenario

You have a lot of email you have to write for the different extracurriculars: movie, running, and math clubs. So, you want to use an LLM to help you write them.


### Exercise 1

You have the following code written, so your LLM can write emails for all your clubs. But it writes a random email. You have no control over what the subject of them will be.

**Your task: Adjust the context to enable you to specify the subject of the email.**

_Goal: Learn how to build reusable contexts/prompts!_


In [8]:
# Build a template for giving context to your LLM; TODO: change the context to take in any subject
context = "Write an email for the {{club_name}} club with the subject: {{subject}}." # double curly braces indicate a variable

my_club_names = ['movie', 'running', 'math']
subjects = [
    'Inviting suggestions for next movie screening',
    'New marathon alert!',
    'Seminar on advanced probability',
]
for i in range(len(my_club_names)):
  # Run your LLM, looping through your different clubs
  club_name = my_club_names[i]
  subject = subjects[i]

  llm_prompt = context
  llm_prompt = llm_prompt.replace("{{club_name}}", club_name)
  llm_prompt = llm_prompt.replace("{{subject}}", subject)

  output = get_completion(llm_prompt, "llama-3.2-90b-text-preview")
  print(output)
  print("--------------")

Subject: Inviting suggestions for next movie screening

Dear Movie Club Members,

I hope this email finds you all well. As we have decided our next movie screening date, it's time for us to choose the film we would like to watch. We are now inviting suggestions from all members for the next movie screening.

We encourage you to suggest any movie, whether it's a classic, a recent release, a foreign film, or any other type that you think the club would enjoy. You can consider various genres such as drama, comedy, thriller, sci-fi, or any other that suits your taste.

If you would like to suggest a movie, please reply to this email with the title of the movie and a brief description (optional) about why you think the club would enjoy it. You can also include a trailer link if available.

We will collect all the suggestions and have a group discussion to decide on our next movie. The deadline for submitting suggestions is [insert date]. We will then send out a voting email to the group, an

### Exercise 2
The emails you're seeing don't sound like you. You sound better, funnier, or at least more like _you_! Good thing you have a few examples of emails you've written before for each club.

**Your task: Given your past emails (provided for you), build a context template to include your past emails and fit a new LLM. Run and compare results.**

_Goal: Observe the impact of more data._

How does it do?

In [14]:
# @markdown #### Setup: Fetch email files

# !wget -q -O "movie_club.txt" "https://drive.google.com/uc?export=download&id=1dWKvcCIp-XwNBZ6heb5k2Y1tbdPl3WTE"
# !wget -q -O "running_club.txt" "https://drive.google.com/uc?export=download&id=1SxeE-0XP4ggjUNYwMD3VQlm_idCHoG9G"
# !wget -q -O "math_club.txt" "https://drive.google.com/uc?export=download&id=12z3EuOauuYc4Y9tdDGENs-_2wmvyErmv"

# print('Got all the emails!')

In [9]:
with open('movie_club.txt', 'r') as f:
  past_movie_club_emails = f.read()
with open('running_club.txt', 'r') as f:
  past_running_club_emails = f.read()
with open('math_club.txt', 'r') as f:
  past_math_club_emails = f.read()

# TODO (same code as above, without your changes for subject in Exercise 1)
context = """Read carefully some of the emails I have written \
and pay attention to my writing style.
<< My old emails start >>
{{old_emails}}
<< My old emails end >>

Write an email for the {{club_name}} club with the subject: {{subject}}.
Copy my writing style."""

my_club_names = ['movie', 'running', 'math']
old_emails_dict = {
    'movie': past_movie_club_emails,
    'running': past_running_club_emails,
    'math': past_math_club_emails,
}
subjects = [
    'Inviting suggestions for next movie screening',
    'New marathon alert!',
    'Seminar on advanced probability',
]
for i in range(len(my_club_names)):
  club_name = my_club_names[i]
  subject = subjects[i]
  old_emails = old_emails_dict[club_name]

  llm_prompt = context
  llm_prompt = llm_prompt.replace("{{old_emails}}", old_emails)
  llm_prompt = llm_prompt.replace("{{club_name}}", club_name)
  llm_prompt = llm_prompt.replace("{{subject}}", subject)

  output = get_completion(llm_prompt, "llama-3.2-90b-text-preview")
  print(output)
  print("--------------")

Subject: Inviting Suggestions for Next Movie Screening 

Lights! Camera! Action!

Movie Maestros and Cinematic Connoisseurs,

Are you ready to shine your spotlight and help pick the next blockbuster for our Movie Club's screening?  We want to hear from YOU! 

As our members, your opinions are the star of the show, and we want to make sure we choose a movie that everyone will love. Whether you're a fan of action-packed adventures or swoon over sappy rom-coms, we want to hear your thoughts and suggestions.

Reply to this email with your top picks for the next movie screening. Are you itching to see the latest superhero film? Craving a classic rom-com to swoon over? Or perhaps you're in the mood for something spooky and thrilling? Share your thoughts and help us put together the ultimate movie night!

To make it even more exciting, the member who suggests the selected movie will receive a FREE ticket and a mystery prize at the next screening! 

Don't be a silent extra – take center stage 

### Exercise 3

So you think you're funny and you prize how funny you are. But what if you can get the LLM to make you funnier?

**Your task: Engineer the prompt such that you can sound funnier than you.**

_Goal: Play with the model, and see where it's easy to manipulate and where it isn't. And that it is not always intuitive._


In [10]:
# TODO (same code as Exercise 2 to get you started, without your changes)

with open('movie_club.txt', 'r') as f:
  past_movie_club_emails = f.read()
with open('running_club.txt', 'r') as f:
  past_running_club_emails = f.read()
with open('math_club.txt', 'r') as f:
  past_math_club_emails = f.read()

context = """Read carefully some of the emails I have written \
and pay attention to my writing style.
<< My old emails start >>
{{old_emails}}
<< My old emails end >>

Write an email for the {{club_name}} club with the subject: {{subject}}.
Make it funnier than my writing style."""

my_club_names = ['movie', 'running', 'math']
old_emails_dict = {
    'movie': past_movie_club_emails,
    'running': past_running_club_emails,
    'math': past_math_club_emails,
}
subjects = [
    'Inviting suggestions for next movie screening',
    'New marathon alert!',
    'Seminar on advanced probability',
]
for i in range(len(my_club_names)):
  club_name = my_club_names[i]
  subject = subjects[i]
  old_emails = old_emails_dict[club_name]

  llm_prompt = context
  llm_prompt = llm_prompt.replace("{{old_emails}}", old_emails)
  llm_prompt = llm_prompt.replace("{{club_name}}", club_name)
  llm_prompt = llm_prompt.replace("{{subject}}", subject)

  output = get_completion(llm_prompt, "llama-3.2-90b-text-preview")
  print(output)
  print("--------------")

Subject: In a Cinematic Crisis: We Need Your Superhero Suggestions!

Greetings, Cinephile Crusaders,

The Movie Club is facing a cinematic catastrophe: we've run out of movie ideas! As our trusted sidekicks, we're turning to you for help. It's time to unleash your inner superhero and save the day (or rather, the screening schedule).

We need your brilliant brains to suggest our next movie masterpiece. Whether you're a die-hard fan of rom-coms, a thrill-seeker of horror flicks, or a serious follower of serious cinema (see what we did there?), we want to hear from you.

Here are the deets:

- No movie is too big or too small (except maybe that one film with the talking pineapple – we're not going there).
- Originality is key, but feel free to suggest classic favorites (we won't judge you, we promise).
- Keep in mind our movie club's guiding mantra: "Laughter is the best genre" (don't worry, we won't actually screen a 3-hour comedy special... or will we?).

Reply to this email with your m

#### Great work! Hope you saw how small programmatic changes to the context for the LLM made sometimes drastic changes and improvements to how the LLM behaved. Now off to do more things with LLMs!